In [1]:
import pandas as pd
import numpy as np

In [45]:
df_qud = pd.read_csv('/content/drive/MyDrive/Planification Projet Capstone/QUD.csv', header=None)
df_qud.columns = ['Occupancy', 'Power Consumption', 'Normalized Power Consumption', 'P(t)-P(t+1)', 'P(t)-P(t-1)', 'Micro-Moment Class']
df_qud

,Occupancy,Power Consumption,Normalized Power Consumption,P(t)-P(t+1),P(t)-P(t-1),Micro-Moment Class
0,1,0.00,-0.002259,0.00,0.00,0
1,1,0.00,-0.002259,-166.43,0.00,1
2,1,166.43,0.007632,0.00,166.43,3
3,1,159.55,0.007223,0.00,0.00,3
4,1,143.27,0.006255,0.00,0.00,3
...,...,...,...,...,...,...
46925,1,-1.64,-0.002357,0.00,0.00,0
46926,1,-10.64,-0.002891,0.00,0.00,0
46927,1,79.22,0.002449,0.00,0.00,0
46928,1,74.38,0.002161,0.00,0.00,0


In [46]:
df_dred = pd.read_csv('/content/drive/MyDrive/Planification Projet Capstone/DRED.csv', header=None)
df_dred.columns = ['Occupancy', 'Power Consumption', 'Normalized Power Consumption', 'P(t)-P(t+1)', 'P(t)-P(t-1)', 'Micro-Moment Class']
df_dred

,Occupancy,Power Consumption,Normalized Power Consumption,P(t)-P(t+1),P(t)-P(t-1),Micro-Moment Class
0,1,0.0,-0.68907,0.0,0.0,0
1,1,0.0,-0.68907,0.0,0.0,0
2,0,0.0,-0.68907,0.0,0.0,0
3,1,0.0,-0.68907,0.0,0.0,0
4,0,0.0,-0.68907,0.0,0.0,0
...,...,...,...,...,...,...
166348,1,0.0,-0.68907,0.0,0.0,0
166349,0,0.0,-0.68907,0.0,0.0,0
166350,0,0.0,-0.68907,0.0,0.0,0
166351,1,0.0,-0.68907,0.0,0.0,0


In [47]:
def sliding_window(dataset, timestep):
    
    dataset = dataset.to_numpy().astype(np.float32)
    timesteps = []
    
    for i in range(dataset.shape[0]):
        if i < dataset.shape[0] - timestep:
            timesteps.append(dataset[i:i+timestep])

    return timesteps

In [48]:
qud_timesteps = sliding_window(df_qud, 3)
dred_timesteps = sliding_window(df_dred, 3)

In [49]:
# TODO: Cette fonction est plutôt lente et pourrait probablement être optimisée
def calculate_covariance_from_timewindow(timesteps, col):

    new_timesteps = np.ones((len(timesteps), timesteps[0].shape[0], timesteps[0].shape[1]+1))

    for i, window in enumerate(timesteps):
        # Calcul de la covariance de la fenêtre
        mean = np.mean(window[:, col])
        max = np.max(window[:, col])
        min = np.min(window[:, col])
        std = np.std(window[:, col])
        covariance = ((max-min) / 100) * (std / mean)

        # Création de la nouvelle fenêtre temporelle avec la covariance
        new_window = np.ones((timesteps[0].shape[0], timesteps[0].shape[1]+1))

        for j, timestep in enumerate(window):
            new_timestep = new_window[j] * np.append(timestep, [1], axis=0)
            new_timestep[-1] = new_timestep[-1] * covariance
            new_window[j] = new_window[j] * new_timestep

        new_timesteps[i] = new_timesteps[i] * new_window
    
    return new_timesteps

In [50]:
new_qud = calculate_covariance_from_timewindow(qud_timesteps, 1)
new_dred = calculate_covariance_from_timewindow(dred_timesteps, 1)

<ipython-input-49-44800b842223>:10: RuntimeWarning: invalid value encountered in float_scalars
  covariance = ((max-min) / 100) * (std / mean)


In [43]:
print(new_qud.shape)
print(new_dred.shape)

(46927, 3, 7)


array([[[ 1.00000000e+00,  0.00000000e+00, -2.25919997e-03, ...,
          0.00000000e+00,  0.00000000e+00,  2.35367549e+00],
        [ 1.00000000e+00,  0.00000000e+00, -2.25919997e-03, ...,
          0.00000000e+00,  1.00000000e+00,  2.35367549e+00],
        [ 1.00000000e+00,  1.66429993e+02,  7.63170002e-03, ...,
          1.66429993e+02,  3.00000000e+00,  2.35367549e+00]],

       [[ 1.00000000e+00,  0.00000000e+00, -2.25919997e-03, ...,
          0.00000000e+00,  1.00000000e+00,  1.17762390e+00],
        [ 1.00000000e+00,  1.66429993e+02,  7.63170002e-03, ...,
          1.66429993e+02,  3.00000000e+00,  1.17762390e+00],
        [ 1.00000000e+00,  1.59550003e+02,  7.22280005e-03, ...,
          0.00000000e+00,  3.00000000e+00,  1.17762390e+00]],

       [[ 1.00000000e+00,  1.66429993e+02,  7.63170002e-03, ...,
          1.66429993e+02,  3.00000000e+00,  1.43789070e-02],
        [ 1.00000000e+00,  1.59550003e+02,  7.22280005e-03, ...,
          0.00000000e+00,  3.00000000e+00,  1.437

In [44]:
print(np.mean(qud_timesteps[0][:, 1]))
print(len(qud_timesteps))

window = qud_timesteps[0]
mean = np.mean(window[:, 1])
max = np.max(window[:, 1])
min = np.min(window[:, 1])
std = np.std(window[:, 1])

covariance = ((max-min) / 100) * (std / mean)
print("Covariance:", covariance)
arr_covariance = [covariance] * 6
arr_covariance = np.expand_dims(arr_covariance, axis=0)
arr_covariance = np.expand_dims(arr_covariance, axis=0)

print("Covariance array:", arr_covariance)

ValueError: ignored